In [1]:
import os
import time
import copy
import glob
import cv2  # ----- OpenCV 라이브러리
import shutil

import torch
import torchvision  # ----- 컴퓨터 비전 (computer vision) 용도의 패키지
import torchvision.transforms as transforms  # ----- 데이터 전처리를 위해 사용되는 패키지
import torchvision.models as models  # ----- 다양한 파이토치 네트워크를 사용할 수 있도록 도와주는 패키지
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

In [8]:
data_path = '/content/drive/MyDrive/FLY_AI/KMK/data/catanddog/train'  # ----- 이미지 데이터가 위치한 경로 지정

transform = transforms.Compose([
    transforms.Resize([256, 256]),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

train_dataset = torchvision.datasets.ImageFolder(
    data_path,
    transform=transform
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32,
    num_workers=8,
    shuffle=True
)

print(len(train_dataset))

192


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [9]:
resnet18 = models.resnet18(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 97.3MB/s]


In [10]:
import torchvision.models as models
resnet18 = models.resnet18()
alexnet = models.alexnet()
vgg16 = models.vgg16()
squeezenet = models.squeezenet1_0()
densenet = models.densenet161()
inception = models.inception_v3()
googlenet = models.googlenet()
shufflenet = models.shufflenet_v2_x1_0()
mobilenet_v2 = models.mobilenet_v2()
mobilenet_v3_large= models.mobilenet_v3_large()
mobilenet_v3_small= models.mobilenet_v3_small()
resnext50_32x4d= models.resnext50_32x4d()
wide_resnet50_2= models.wide_resnet50_2()
mnasnet = models.mnasnet1_0()

/usr/local/lib/python3.11/dist-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/googlenet.py:47: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [11]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

set_parameter_requires_grad(resnet18, True)

In [12]:
resnet18.fc = nn.Linear(512, 2)

In [13]:
for name, param in resnet18.named_parameters():
    if param.requires_grad == True:
      print(name, param.requires_grad)

fc.weight True
fc.bias True


In [14]:
model = models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.fc = torch.nn.Linear(512, 2)
for param in model.fc.parameters():
    param.requires_grad = True

optimizer = torch.optim.Adam(model.fc.parameters())
cost = torch.nn.CrossEntropyLoss()
print(model)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [15]:
# def train_model(model, dataloaders, criterion, optimizer, devicem, num_epochs=13,
#                 is_train=True):
#     since = time.time()
#     acc_history = []
#     loss_history = []
#     best_acc = 0.0

#     for epoch in range(num_epochs):
#         print('Epoch {}/{}'.format(epoch, num_epochs - 1))
#         print('-' * 10)

#         running_loss = 0.0
#         running_corrects = 0.0

#         for inputs, labels in dataloaders:
#             inputs = inputs.to(device)
#             labels = labels.to(device)

#             model.to(device)
#             optimizer.zero_grad()
#             outputs = model(inputs) # Propagation
#             _, preds = torch.max(outputs, 1)
#             loss.backward()
#             optimizer.step()

#             running_loss += loss.item() * inputs.size(0)
#             running_corrects += torch.sum(preds == labels.data)

#         epoch_loss = running_loss / len(dataloaders.dataset)
#         epoch_acc = running_corrects.double() / len(dataloaders.dataset)
#         print(f"Loss: {epoch_loss} Acc: {epoch_acc}")

#         if epoch_acc > best_acc:
#             best_acc = epoch_acc

#         acc_history.append(epoch_acc.item())

#         loss_history.append(epoch_loss)
#         torch.save(model.state_dict(), os.path.join('/content/drive/MyDrive/FLY_AI/KMK/data/catanddog/', '{0:0=2d}.pth'.format(epoch)))
#         print()

#     time_elapsed = time.time() - since
#     print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
#     print('Best val Acc: {:4f}'.format(best_acc))
#     return model, acc_history, loss_history

In [18]:
def train_model(model, dataloaders, criterion, optimizer, device, num_epochs=13, is_train=True):
    since = time.time()
    acc_history = []
    loss_history = []
    best_acc = 0.0

    save_dir = '/content/drive/MyDrive/FLY_AI/KMK/data/catanddog/'
    os.makedirs(save_dir, exist_ok=True)

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        running_loss = 0.0
        running_corrects = 0

        if is_train:
            model.train()  # 학습 모드
        else:
            model.eval()  # 평가 모드

        for inputs, labels in dataloaders:
            inputs = inputs.to(device)
            labels = labels.to(device)

            model.to(device)
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)  # Propagation
            _, preds = torch.max(outputs, 1)

            # Calculate loss
            loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data).item()

        epoch_loss = running_loss / len(dataloaders.dataset)
        epoch_acc = running_corrects / len(dataloaders.dataset)
        print(f"Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        if epoch_acc > best_acc:
            best_acc = epoch_acc

        acc_history.append(epoch_acc)
        loss_history.append(epoch_loss)

        # Save model checkpoint
        torch.save(model.state_dict(), os.path.join(save_dir, '{0:0=2d}.pth'.format(epoch)))
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    return model, acc_history, loss_history

In [19]:
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print('\t', name)

optimizer = optim.Adam(params_to_update)

	 fc.weight
	 fc.bias


In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
train_acc_hist, train_loss_hist = train_model(model, train_loader, criterion, optimizer, device, num_epochs=13)

Epoch 0/12
----------
Loss: 0.2440 Acc: 0.9115

Epoch 1/12
----------
Loss: 0.2721 Acc: 0.9479

Epoch 2/12
----------
Loss: 0.2239 Acc: 0.9479

Epoch 3/12
----------
Loss: 0.1700 Acc: 0.9479

Epoch 4/12
----------
Loss: 0.1537 Acc: 0.9531

Epoch 5/12
----------
Loss: 0.1428 Acc: 0.9479

Epoch 6/12
----------
Loss: 0.1312 Acc: 0.9479

Epoch 7/12
----------
Loss: 0.1199 Acc: 0.9479

Epoch 8/12
----------
Loss: 0.1193 Acc: 0.9479

Epoch 9/12
----------
Loss: 0.1241 Acc: 0.9531

Epoch 10/12
----------
Loss: 0.1037 Acc: 0.9635

Epoch 11/12
----------
Loss: 0.1228 Acc: 0.9531

Epoch 12/12
----------
Loss: 0.0862 Acc: 0.9583

Training complete in 6m 41s
Best val Acc: 0.963542


ValueError: too many values to unpack (expected 2)

In [21]:
test_path = '/content/drive/MyDrive/FLY_AI/KMK/data/catanddog/test'

transform = transforms.Compose(
    [
    transforms.Resize([224]),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    ])

test_dataset = torchvision.datasets.ImageFolder(
    test_path,
    transform=transform
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=32,
    num_workers=1,
    shuffle=True
)

print(len(test_dataset))

98


In [ ]:
def eval_model(model, dataloaders, device):
    since = time.time()
    acc_history = []
    best_acc = 0.0

    saved_models = glob.glob('/content/drive/MyDrive/FLY_AI/KMK/data/catanddog/*.pth')
    saved_models.sort()
    print('saved_model', saved_models)

    for model_path in saved_models:
        print("Loading model", model_path)
        model.load_state_dict(torch.load(model_path))
        model.eval()
        model.to(device)
        running_corrects = 0

        for inputs, labels in dataloaders:
            inputs = inputs.to(device)
            labels = labels.to(device)

            with torch.no_grad():
                outputs = model(inputs)


            _, preds = torch.max(outputs.data, 1)
            preds[preds >= 0.5] = 1
            preds[preds < 0.5] = 0
            running_corrects += preds.eq(labels.cpu()).int.sum()

        epoch_acc = running_corrects.double() / len(dataloaders.dataset)
        print(f"Acc: {epoch_acc}")